In [ ]:
# Import arcpy module
import arcpy
from arcpy.sa import *
import os

# Set environment settings
arcpy.env.workspace = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2021\DSM"  # Replace with the folder containing your TIFF files
arcpy.env.overwriteOutput = True

# Path to the point shapefile
point_shapefile = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\RIP_ptos.shp"  # Replace with your point shapefile

# Output folder for the kriging results
output_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2021\MS\DTM_PTOS"  # Replace with your desired output folder

# List all TIFF files in the input folder
tiff_files = arcpy.ListRasters("*.tif")

# Loop through each TIFF file in the folder
for tiff_file in tiff_files:
    print(f"Processing {tiff_file}...")

    # Step 1: Extract the projection from the original TIFF file
    desc = arcpy.Describe(tiff_file)
    tif_projection = desc.spatialReference

    # Step 2: Extract pixel values to points, ignoring NoData and values below 0
    arcpy.CheckOutExtension("Spatial")
    extracted_points = os.path.join(arcpy.env.workspace, "points_with_values.shp")
    
    # Extract the pixel values
    arcpy.sa.ExtractValuesToPoints(point_shapefile, tiff_file, extracted_points, "NONE", "VALUE_ONLY")
    print(f"Extracted pixel values from {tiff_file} to {extracted_points}")

    # Step 3: Select only valid points (ignore NoData and values below 0)
    valid_points = os.path.join(arcpy.env.workspace, "valid_points.shp")
    
    # NoData typically corresponds to null values, so we filter values >= 0
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=extracted_points,
        selection_type="NEW_SELECTION",
        where_clause="RASTERVALU IS NOT NULL AND RASTERVALU >= 0"
    )
    
    # Copy the selected points (valid values) to a new shapefile
    arcpy.management.CopyFeatures(extracted_points, valid_points)
    print(f"Filtered valid points (RASTERVALUE >= 0 and not NoData) saved to {valid_points}")

    # Step 4: Check if there are any valid points remaining, if not, skip to the next raster
    point_count = int(arcpy.management.GetCount(valid_points)[0])
    if point_count == 0:
        print(f"No valid points found for {tiff_file}. Skipping...")
        continue

    # Step 5: Define Kriging model parameters
    semivariogram_type = "Spherical"  # Example semivariogram type, could be "Exponential", "Gaussian", etc.
    lag_size = 10  # Example lag size, adjust as needed
    major_range = 100  # Example range, adjust as needed
    partial_sill = 0.5  # Example partial sill, adjust as needed
    nugget = 0.1  # Example nugget, adjust as needed

    # Create the Kriging model
    kriging_model = KrigingModelOrdinary(
        semivariogram_type,
        lag_size,
        major_range,
        partial_sill,
        nugget
    )

    # Step 6: Perform Kriging on the valid points (filtered)
    kriging_output = os.path.join(output_folder, f"{os.path.splitext(tiff_file)[0]}_krig.tif")

    kriging_result = Kriging(
        in_point_features=valid_points,
        z_field="RASTERVALU",  # This field contains the extracted pixel values
        kriging_model=kriging_model,
        cell_size=desc.meanCellHeight  # Use the original cell size from the TIFF file
    )

    # Save the Kriging output raster
    kriging_result.save(kriging_output)
    print(f"Kriging raster saved to {kriging_output}")

    # Step 7: Project the Kriging output to match the original TIFF's projection
    projected_output = os.path.join(output_folder, f"{os.path.splitext(tiff_file)[0]}_krig_projected.tif")
    arcpy.management.ProjectRaster(
        in_raster=kriging_output,
        out_raster=projected_output,
        out_coor_system=tif_projection,
        resampling_type="BILINEAR"
    )
    print(f"Projected Kriging raster saved to {projected_output}")

    # Step 8: Clean up - delete the temporary point shapefiles with extracted and valid points
    arcpy.management.Delete(extracted_points)
    arcpy.management.Delete(valid_points)
    print(f"Deleted temporary shapefiles for {tiff_file}")

    # Clean up temporary files generated during the process
    arcpy.CheckInExtension("Spatial")

print("Processing completed.")


Processing RIP_20210729_1510_DSM.tif...
Extracted pixel values from RIP_20210729_1510_DSM.tif to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2021\DSM\points_with_values.shp
Filtered valid points (RASTERVALUE >= 0 and not NoData) saved to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\RIP\2021\DSM\valid_points.shp


# WIDTH OVER HEIGHT

In [5]:
# Import arcpy module
import arcpy
from arcpy.sa import *
import os

# Set environment settings
arcpy.env.workspace = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\WWF\MS-ALTUMPT\WIDTH"  # Replace with your folder containing TIFF files
arcpy.env.overwriteOutput = True

# Specify the reference TIFF file
reference_tif = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\WWF\MS-ALTUMPT\WWF_2022_HEIGHT_ok.tif"  # Replace with your reference TIFF file

# Specify the output folder for saving the result TIFF files
output_folder = r"C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\WWF\MS-ALTUMPT\WIDTH_OVER_HEIGHT"  # Replace with your output folder path

# Check if the output folder exists, if not, create it
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List all TIFF files in the folder (excluding the reference file)
tiff_files = arcpy.ListRasters("*.tif")

# Loop through each TIFF file in the folder
for tiff_file in tiff_files:
    # Make sure we are not dividing the reference TIFF by itself
    if os.path.abspath(tiff_file) == os.path.abspath(reference_tif):
        continue
    
    print(f"Processing {tiff_file}...")

    # Step 1: Convert both TIFF files into raster objects
    numerator_raster = Raster(tiff_file)
    divisor_raster = Raster(reference_tif)

    # Step 2: Perform the division, handling the case where the divisor is zero
    # If the divisor is zero, the result will be zero.
    result_raster = Con(divisor_raster != 0, numerator_raster / divisor_raster, 0)

    # Step 3: Save the result to the second folder with "_WOVERH" added to the name
    output_filename = os.path.splitext(os.path.basename(tiff_file))[0] + "_WOVERH.tif"
    output_filepath = os.path.join(output_folder, output_filename)
    result_raster.save(output_filepath)

    print(f"Saved result to {output_filepath}")

print("Processing completed.")


Processing WWF_20220710_1137_6band_NDVI_canopy_FC_OK_width.tif...
Saved result to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\WWF\MS-ALTUMPT\WIDTH_OVER_HEIGHT\WWF_20220710_1137_6band_NDVI_canopy_FC_OK_width_WOVERH.tif
Processing WWF_20220710_1250_6band_NDVI_canopy_FC_OK_width.tif...
Saved result to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\WWF\MS-ALTUMPT\WIDTH_OVER_HEIGHT\WWF_20220710_1250_6band_NDVI_canopy_FC_OK_width_WOVERH.tif
Processing WWF_20220710_1515_6band_NDVI_canopy_FC_OK_width.tif...
Saved result to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\WWF\MS-ALTUMPT\WIDTH_OVER_HEIGHT\WWF_20220710_1515_6band_NDVI_canopy_FC_OK_width_WOVERH.tif
Processing WWF_20220714_1135_6band_NDVI_canopy_FC_OK_width.tif...
Saved result to C:\Users\14352\Desktop\01_TREX-UAV-AggieAir-metadata\WWF\MS-ALTUMPT\WIDTH_OVER_HEIGHT\WWF_20220714_1135_6band_NDVI_canopy_FC_OK_width_WOVERH.tif
Processing WWF_20220714_1249_6band_NDVI_canopy_FC_OK_width.tif...
Saved result to C:\Users\1